In [109]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
import seaborn as sns

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

from sklearn.model_selection import cross_val_score
import lightgbm as lgb

from sklearn.model_selection import train_test_split

In [ ]:
### находим индексы фейковых строк в тесте

In [11]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

In [15]:
df_test.drop(['ID_code'], axis=1, inplace=True)
df_test = df_test.values

unique_samples = []
unique_count = np.zeros_like(df_test)
for feature in range(df_test.shape[1]):
    _, index_, count_ = np.unique(df_test[:, feature], return_counts=True, return_index=True)
    unique_count[index_[count_ == 1], feature] += 1

# Samples which have unique values are real the others are fake
real_samples_indexes = np.argwhere(np.sum(unique_count, axis=1) > 0)[:, 0]
synthetic_samples_indexes = np.argwhere(np.sum(unique_count, axis=1) == 0)[:, 0]

print(len(real_samples_indexes))
print(len(synthetic_samples_indexes))

100000
100000


In [ ]:
### добавляем новые фичи

In [238]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

In [184]:
orig_col_list = df_train.columns[:-6]

In [241]:
df_train, target = df_train.iloc[:, 2:], df_train['target']

test_id_code = df_test['ID_code']
df_test.drop(['ID_code'], axis=1, inplace=True)

In [200]:
orig_len = df_train.shape[0]

In [243]:
X_train, X_test = df_train.values, df_test.values

In [244]:
X_train.shape, X_test.shape

((200000, 200), (200000, 200))

In [194]:
X_test_real = X_test[real_samples_indexes]
X_all = np.vstack([X_train, X_test_real])
X_all_df = pd.DataFrame(X_all)

In [195]:
%%time

F_all = np.zeros_like(X_all, dtype=int)
F_counts = np.zeros_like(X_all, dtype=int)

for i, c in enumerate(X_all_df.columns):
    a = X_all_df.groupby(c)[c].transform('count').values
    F_all[:,i] = (a == 1).astype(int)
    F_counts[:,i] = a / X_all_df.shape[0]

CPU times: user 13.8 s, sys: 721 ms, total: 14.5 s
Wall time: 14.6 s


In [210]:
def add_features(X, F, F_c):
    """
    
    """
    return np.hstack([X, F[:len(X)], X*F[:len(X)] ])

In [245]:
def add_features_v2(X, F):
    """
    
    """
    return np.hstack([X, F, X*F[:len(X)]]) ##, np.mean(X, axis=1)[:, None], np.std(X, axis=1)[:, None]])

In [247]:
X_train = add_features_v2(X_train, F_all[:orig_len])
X_test_real = add_features_v2(X_test[real_samples_indexes], F_all[orig_len:])

X_test_all = np.zeros((len(X_test), X_test_real.shape[1]))
X_test_all[real_samples_indexes] = X_test_real
X_test = X_test_all

In [248]:
X_train.shape, X_test.shape

((200000, 600), (200000, 600))

In [249]:
features_diff = ['var_0', 'var_2', 'var_6', 'var_9', 'var_12', 'var_13', 'var_22', 'var_26', 'var_40',
                 'var_44', 'var_53','var_76', 'var_80', 'var_81', 'var_86', 'var_94', 'var_95', 'var_108', 
                 'var_109', 'var_110', 'var_112', 'var_115', 'var_123', 'var_131', 'var_133', 'var_139', 
                 'var_146', 'var_148', 'var_164', 'var_165', 'var_166','var_174', 'var_177', 'var_179', 
                 'var_131', 'var_180', 'var_184', 'var_188', 'var_131', 'var_190', 'var_191', 'var_195', 'var_198']

##features_diff = orig_col_list

df_train['dif_mean'] = df_train[features_diff].mean(axis = 1)
df_train['dif_min'] = df_train[features_diff].min(axis = 1)
df_train['dif_max'] = df_train[features_diff].max(axis = 1)

df_train['dif_std'] = df_train[features_diff].std(axis = 1)
df_train['dif_median'] = df_train[features_diff].median(axis = 1)
df_train['dif_sum'] = df_train[features_diff].sum(axis = 1)

df_test['dif_mean'] = df_test[features_diff].mean(axis = 1)
df_test['dif_min'] = df_test[features_diff].min(axis = 1)
df_test['dif_max'] = df_test[features_diff].max(axis = 1)

df_test['dif_std'] = df_test[features_diff].std(axis = 1)
df_test['dif_median'] = df_test[features_diff].median(axis = 1)
df_test['dif_sum'] = df_test[features_diff].sum(axis = 1)

In [250]:
new_feature_list = ['dif_mean', 'dif_min', 'dif_max', 'dif_std', 'dif_median', 'dif_sum']

In [252]:
X_train = np.hstack([X_train, df_train[new_feature_list].values])
X_test = np.hstack([X_test, df_test[new_feature_list].values])

In [253]:
X_train.shape, X_test.shape

((200000, 606), (200000, 606))

### Модель

In [156]:
param = {
    'bagging_freq': 5,
    'bagging_fraction': 0.4, ##0.2,
    'boost_from_average':'false',
    'boost': 'gbdt',
    'feature_fraction': 0.05, ##0.1,
    'learning_rate': 0.01,
    'max_depth': -1,  
    'metric':'auc',
    'min_data_in_leaf': 80,
    'min_sum_hessian_in_leaf': 10.0,
    'num_leaves': 13, ##15,
    'tree_learner': 'serial',
    'objective': 'binary', 
    'verbosity': 1,
    'device': 'cpu',
    'gpu_platform_id': 0,
    'gpu_device_id': 0,
    'max_bin': 255,
}


In [254]:
train_df = pd.DataFrame(X_train)
test_df = pd.DataFrame(X_test)

X_train, X_test, y_train, y_test = train_test_split(train_df, target, test_size=0.2, random_state=42)

In [255]:
trn_data = lgb.Dataset(X_train, label=y_train)
val_data = lgb.Dataset(X_test, label=y_test)

num_round = 1000000
clf = lgb.train(param, trn_data, num_round, valid_sets = [trn_data, val_data], verbose_eval=1000, early_stopping_rounds = 3000)

Training until validation scores don't improve for 3000 rounds
[1000]	training's auc: 0.902521	valid_1's auc: 0.884347
[2000]	training's auc: 0.915696	valid_1's auc: 0.892994
[3000]	training's auc: 0.924359	valid_1's auc: 0.897504
[4000]	training's auc: 0.931471	valid_1's auc: 0.900698
[5000]	training's auc: 0.937939	valid_1's auc: 0.903319
[6000]	training's auc: 0.943631	valid_1's auc: 0.905306
[7000]	training's auc: 0.948798	valid_1's auc: 0.906803
[8000]	training's auc: 0.953414	valid_1's auc: 0.908251
[9000]	training's auc: 0.957656	valid_1's auc: 0.909391
[10000]	training's auc: 0.961461	valid_1's auc: 0.910183
[11000]	training's auc: 0.964983	valid_1's auc: 0.910972
[12000]	training's auc: 0.968203	valid_1's auc: 0.911625
[13000]	training's auc: 0.971107	valid_1's auc: 0.912199
[14000]	training's auc: 0.973819	valid_1's auc: 0.912667
[15000]	training's auc: 0.976253	valid_1's auc: 0.913073
[16000]	training's auc: 0.978507	valid_1's auc: 0.913354
[17000]	training's auc: 0.980687	v

In [ ]:
Training until validation scores don't improve for 3000 rounds
[1000]	training's auc: 0.900367	valid_1's auc: 0.882546
[2000]	training's auc: 0.918746	valid_1's auc: 0.894686
[3000]	training's auc: 0.928791	valid_1's auc: 0.899364
[4000]	training's auc: 0.935792	valid_1's auc: 0.901126
[5000]	training's auc: 0.941591	valid_1's auc: 0.901935
[6000]	training's auc: 0.946818	valid_1's auc: 0.902299
[7000]	training's auc: 0.951828	valid_1's auc: 0.90243
[8000]	training's auc: 0.956545	valid_1's auc: 0.902584
[9000]	training's auc: 0.960866	valid_1's auc: 0.90257
[10000]	training's auc: 0.964893	valid_1's auc: 0.902313
Early stopping, best iteration is:
[7656]	training's auc: 0.954916	valid_1's auc: 0.902748

In [256]:
preds = clf.predict(test_df)

sub_df = pd.DataFrame({"ID_code":  test_id_code.values})
sub_df["target"] = preds
sub_df.to_csv("submission.csv", index=False)